Film Agent

# Building an AI Film Analysis Agent

## Business Objective

This notebook guides you through building an AI agent specialized in film analysis using AWS services. Before creating the agent itself, we first use Bedrock Data Automation (BDA) to analyze video content and extract key information. The agent then combines this analysis with a knowledge base of film information and celebrity recognition to automatically identify and analyze film content.

Media companies need to efficiently process and identify film content across their large libraries. By deploying an AI agent, tasks that traditionally required hours of manual review - like identifying clips, recognizing actors, and extracting metadata - can be automated. This automation helps production teams work more efficiently and helps distribution teams better manage their content.

## Main Steps

### 1. Video Analysis with BDA
We begin by using Bedrock Data Automation to analyze our video content. This involves:
- Setting up a BDA project with specific video analysis configurations
- Uploading video content for processing
- Extracting rich metadata including transcripts, summaries, and scene analysis
This initial analysis provides the structured data our agent will use to understand and identify film content.

### 2. Knowledge Base Creation 
Next, we build a knowledge base containing film information. We upload film data to S3, configure vector search capabilities, and synchronize the data. This knowledge base gives our agent the background information it needs to match video content with specific films.

### 3. Film Agent Development
At this stage, we create the AI agent itself using Amazon Bedrock. We configure the agent with specific instructions for film analysis and connect it to the knowledge base. The agent uses foundation models to understand and analyze film content, enabling it to match clips to their source films.

### 4. Celebrity Detection Integration
We enhance the agent's capabilities by adding celebrity detection. Using a Lambda function and DynamoDB, we enable the agent to recognize actors in video clips and retrieve information about their roles. This feature helps confirm film identification through cast appearance.

### 5. Testing and Validation
We thoroughly test the agent's capabilities, including:
- Searching the knowledge base for film information
- Detecting and identifying celebrities in video clips  
- Running complete film identification workflows

These tests ensure the agent can accurately identify films using both content analysis and cast recognition.

### 6. Production Preparation
Finally, we prepare the agent for production use by:
- Creating a stable agent alias
- Saving all configuration information
- Setting up proper resource management

This notebook shows you how to combine AWS AI services to create a practical solution for film content identification and analysis. The resulting agent helps media companies automate content processing while maintaining accuracy and consistency.


#### > Setup

In [ ]:
!pip freeze | grep boto3

### Analyze and extract video

Before building our agent, we first need to analyze the video content to extract meaningful information. We'll use Amazon Bedrock Data Automation (BDA) which provides powerful AI capabilities for video analysis.

In this section, we first set up the required AWS services and then create a BDA project configured for video analysis. The project will extract several types of information from our video:
- A comprehensive video summary
- Scene-by-scene analysis
- Full audio transcript 
- Text detected in the video
- Scene classifications

This extracted information will serve as the foundation for our agent's ability to identify and analyze film content. Let's start by configuring the necessary AWS clients and resources.

First of all let's initialize the environment for video analysis using AWS services. The following setup will prepare the variables to create a Bedrock Data Automation project, upload a video file, and extract insights from it using AWS's AI services.

In [ ]:
import boto3
import json
import uuid
import sagemaker
import uuid

boto_session = boto3.session.Session()
sess = sagemaker.Session(boto_session=boto_session)
if sess.default_bucket():
    bucket = sess.default_bucket()
else:
    bucket = "<YOUR-BUCKET-NAME>" # Provide your own bucket
region = sess.boto_region_name
prefix = "media-operations-agent-claude"

bda_client = boto_session.client('bedrock-data-automation')
bda_runtime_client = boto_session.client('bedrock-data-automation-runtime')
s3_client = boto_session.client('s3')

#access account id
sts_client = boto_session.client('sts')
account_id = sts_client.get_caller_identity()["Account"]

default_profile_arn = f"arn:aws:bedrock:{region}:{account_id}:data-automation-profile/us.data-automation-v1"

### Prepare the sample video

In [ ]:
video_file = 'NetflixMeridian.mp4'
!curl "https://ws-assets-prod-iad-r-pdx-f3b3f9f1a7d6a3d0.s3.us-west-2.amazonaws.com/7db2455e-0fa6-4f6d-9973-84daccd6421f/Netflix_Open_Content_Meridian.mp4" --output NetflixMeridian.mp4

In [ ]:
s3_key = f"{prefix}/{video_file}"

s3_client.upload_file(video_file, bucket, s3_key)
film_video_s3_path = f"s3://{bucket}/{s3_key}"

### Create a BDA project

To analyze our film content, we first need to create a Bedrock Data Automation (BDA) project. This project will define what information we want to extract from our video and how it should be processed.

A BDA project organizes both standard and custom output configurations, making it reusable across multiple videos that need similar analysis. For our film analysis agent, we'll configure the project to extract:
- Full video summary: A comprehensive overview of the video content
- Scene summaries: Detailed analysis of individual scenes
- Audio transcript: Complete transcription of spoken content
- Text detection: Identification of text visible in the video

In the code below, we create a BDA project with these configurations. Each extracted element will later help our agent:
- Understand the film's plot and content (summaries)
- Follow dialogue and narration (transcript)
- Identify on-screen text that might indicate the film's source

For a complete API reference for creating a BDA project, refer to this [document](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-data-automation/client/create_data_automation_project.html).

The BDA project creation returns a project ARN (Amazon Resource Name), which we'll store in video_project_arn. We'll need this unique identifier later to invoke the video analysis task and track its progress.


In [ ]:
response = bda_client.create_data_automation_project(
    projectName=f'{prefix}-{str(uuid.uuid4())[0:4]}',
    projectDescription='Media operations agent',
    projectStage='DEVELOPMENT',
    standardOutputConfiguration={
        'video': {
            'extraction': {
                'category': {
                    'state': 'ENABLED',
                    'types': ['TEXT_DETECTION','TRANSCRIPT'],
                },
                'boundingBox': {
                    'state': 'DISABLED',
                }
            },
            'generativeField': {
                'state': 'ENABLED',
                'types': ['VIDEO_SUMMARY','CHAPTER_SUMMARY','IAB'],
            }
        }
    }
)

video_project_arn = response.get("projectArn")
print("BDA video project ARN:", video_project_arn)

### Extract analysis from video

Now that our video is in S3, we can start the BDA analysis process. We'll use the invoke_data_automation_async API to launch an asynchronous analysis task. This task will process our video using the configurations we defined in our BDA project.

The code below:
1. Configures the input (our S3 video) and output locations
2. References our BDA project for analysis settings
3. Launches the analysis task
4. Returns an invocation ARN that we'll use to track the task's progress

This analysis will extract all the elements we need for our agent:
- Video summaries
- Scene analysis
- Audio transcripts
- Detected text

Note that this is an asynchronous process - we'll monitnitor its progress in the next section. In a production environment, you might want to use event notifications instead of polling, but for this demonstration, we'll actively track the task's status.


In [ ]:
response = bda_runtime_client.invoke_data_automation_async(
    inputConfiguration={
        's3Uri': f's3://{bucket}/{s3_key}'
    },
    outputConfiguration={
        's3Uri': f's3://{bucket}/{prefix}/outputs'
    },
    dataAutomationConfiguration={
        'dataAutomationProjectArn': video_project_arn,
        'stage': 'DEVELOPMENT'
    },
    notificationConfiguration={
        'eventBridgeConfiguration': {
            'eventBridgeEnabled': False
        }
    },
    dataAutomationProfileArn=default_profile_arn
)

invocation_arn = response.get("invocationArn")
print("BDA task started:", invocation_arn)

### Wait

Now that we've launched our BDA analysis task, we need to monitor its progress until completion. Since this is an asynchronous process, we'll implement a polling mechanism that checks the task status every few seconds.

The code below:
1. Creates a monitoring loop that calls get_data_automation_status
2. Updates us with timestamps and current status
3. Continues until the job reaches a final state (Success, ServiceError, or ClientError)
4. Retrieves the S3 location of our analysis results upon completion

The analysis typically takes 5-10 minutes to process the video. While polling works well for this demonstration, in a production environment you might prefer using AWS EventBridge notifications to trigger your next steps automatically when the analysis completes.


In [ ]:
import time
from IPython.display import clear_output
from datetime import datetime

status, status_response = None, None
while status not in ["Success","ServiceError","ClientError"]:
    status_response = bda_runtime_client.get_data_automation_status(
        invocationArn=invocation_arn
    )
    status = status_response.get("status")
    clear_output(wait=True)
    print(f"{datetime.now().strftime('%H:%M:%S')} : BDA video task: {status}")
    time.sleep(5)

output_config = status_response.get("outputConfiguration",{}).get("s3Uri")
print("Ouput configureation file:", output_config)

### Access the BDA Analysis Result

Now that our video analysis is complete, we'll retrieve and structure the results for our agent to use. The BDA analysis provides rich information about our video content that will be crucial for film identification.

First, we'll get the job metadata from the configuration file, which tells us where to find our detailed analysis results. Then, we'll fetch and organize this analysis data into a format our agent can effectively use.

The BDA analysis provides comprehensive information about our video:
- A detailed overall summary of the film's content
- Scene-by-scene analysis and transitions
- Complete audio transcription
- Technical metadata about the video file

This extracted information forms the foundation of our agent's ability to identify and analyze films. We'll store this data in a structured format that allows our agent to:
- Match plot details against our knowledge base
- Analyze scene content for film identification
- Use transcripts to confirm dialogue matching
- Leverage technical details for verification

We'll use the IPython.display JSON viewer to explore this rich dataset, making it easier to understand the structure and content of our analysis results.


In [ ]:
def read_json_on_s3(s3_uri, s3_client):
    # Parse s3 bucket and key from s3 uri
    s3_bucket = s3_uri.split('/')[2]
    s3_key = s3_uri.replace(f's3://{s3_bucket}/','')
    
    # Read BDA output_config file on S3
    response = s3_client.get_object(Bucket=s3_bucket, Key=s3_key)
    file_content = response['Body'].read().decode('utf-8')  # Read the content and decode it to a string
    # Convert the content to JSON
    return json.loads(file_content)

The BDA output configuration file provides the S3 location where our analysis results are stored. First, let's read and display this configuration file to locate our results:


In [ ]:
config_data = read_json_on_s3(output_config,s3_client)
print(json.dumps(config_data, indent=4))

As we can see, this configuration file contains metadata about our BDA analysis, including the job ID, status, and most importantly, the S3 path to our full analysis results. Now let's retrieve those detailed results:


In [ ]:
from IPython.display import JSON

result_uri = config_data["output_metadata"][0]["segment_metadata"][0]["standard_output_path"]
result_data = read_json_on_s3(result_uri,s3_client)

JSON(result_data)

Let's organize the most relevant information from our BDA analysis into a structured format for our agent, combining the visual summary, audio transcription, and technical metadata:


In [ ]:
video_analysis = {
    "visual_summary": result_data["video"]["summary"],
    "audio_transcription":result_data["video"]["transcript"]["representation"]["text"],
    "metadata":result_data["metadata"]
    
}
video_analysis

## Setup Film Agent

Now that we have our video analysis results, let's create an AI agent using Amazon Bedrock Agents. Bedrock Agents provide a powerful framework for building AI applications that can understand natural language, process information, and take actions using foundation models.

In our case, we'll create an agent specialized in film analysis that can:
- Process video content analysis
- Search through film databases
- Recognize celebrities in footage
- Combine multiple sources of information to identify films

### Why Use an Agent?
Agents excel at orchestrating complex tasks that require multiple steps and different types of analysis. For film identification, our agent will:
1. Analyze the video content we extracted with BDA
2. Query a knowledge base of film information
3. Use celebrity recognition to identify actors
4. Combine these insights to make accurate film identifications

This automated approach helps media companies process content more efficiently than manual review while maintaining high accuracy.

In the following sections, we'll:
- Initialize the AWS environment for our agent
- Configure the agent with specific film analysis instructions
- Set up foundation models for the agent to use

For more information about Amazon Bedrock Agents and their capabilities, see the [Amazon Bedrock Agents documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/agents.html).

Let's start by setting up the necessary AWS clients and configurations:


In [ ]:
import os
import time
import sys

account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'us.anthropic.claude-3-5-sonnet-20241022-v2:0',
]

### Configure Resource Names

Before creating our agent, we need to set up unique identifiers for all the AWS resources we'll use. Following AWS naming best practices, we'll create standardized names that clearly identify our resources while maintaining consistency across our solution.

We'll configure names for:
1. The agent and its Lambda function
   - Using region and account identifiers for uniqueness
   - Following a consistent prefix pattern for related resources

2. Knowledge base resources
   - Creating a descriptive name and description
   - Setting up an S3 bucket for film information storage

3. Cast member database
   - Defining DynamoDB table name and keys
   - Maintaining naming consistency with other components

This standardized naming approach helps us:
- Easily identify related resources
- Maintain clear resource organization
- Enable proper resource management
- Support future scaling of our solution

Let's define these resource names:


In [ ]:
agent_name = f"f-agent-{agent_suffix}"
lambda_name = f"fn-f-agent-{agent_suffix}"

agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'

knowledge_base_name = f'{agent_name}-02'

suffix = f"{region}-{account_id}"

knowledge_base_description = "KB containing information of all the films"
bucket_name = f'{agent_name}-{suffix}'

cast_table = f'cast-table-{agent_suffix}'
cast_pk = 'id'
cast_sk = 'name'
env_args = [cast_table, cast_pk, cast_sk]
bucket_name

### Importing helper functions

On following section, we're adding `bedrock_agent_helper.py` and `knowledge_base_helper` on Python path, so the files can be recognized and their functionalities can be invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py` and `knowledge_base_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:
- `create_agent`: Create a new agent and respective IAM roles
- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
- `create_agent_alias`: Create an alias for this agent
- `invoke`: Execute agent

On `knowledge_bases.py`:
- `create_or_retrieve_knowledge_base`: Create Knowledge Base on Amazon Bedrock if it doesn't exist or get info about previous created.
- `synchronize_data`: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [ ]:
import sys
sys.path.insert(0, '..')

from helper.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from helper.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## Create and Synchronize Knowledge Base

For our film analysis agent to effectively identify movies, it needs access to accurate film information. Amazon Bedrock Knowledge Bases provides a fully managed solution that allows our agent to search and retrieve film details using natural language processing. 

### Why Use a Knowledge Base?
Our agent needs to match video content with specific films by:
- Finding films with similar plots
- Verifying director and cast information
- Confirming production details
- Cross-referencing scene descriptions

A knowledge base enables these capabilities by:
- Storing structured film information
- Providing fast, accurate search capabilities
- Enabling natural language queries
- Supporting context-aware responses

### What We'll Build
In this section, we'll:
1. Create a knowledge base specialized for film information
2. Upload our film data to S3
3. Configure vector search capabilities for efficient matching
4. Synchronize the data to make it available to our agent

This knowledge base will serve as our agent's reference library, helping it accurately identify films based on the video analysis we performed earlier. Let's start by creating the knowledge base:


**This creation process can take several minutes.**

In [ ]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

### Upload Film Documents to S3

Before our knowledge base can be used, we need to upload our film information to S3. We'll use the AWS S3 sync command to efficiently transfer our film documents to the cloud.

Our film information is stored in two types of files:
- Text files (.txt) containing film plots, descriptions, and details
- Metadata files (.json) with structured information about directors, release dates, and other film attributes

First, let's clean up any notebook checkpoints to ensure we only upload the essential files:


In [ ]:
!rm -rf `find -type d -name .ipynb_checkpoints`

Now we'll sync our film documents to the S3 bucket we created earlier. The sync command will:
- Upload our film information files
- Maintain the file structure
- Only transfer new or modified files

This uploaded content will form the basis of our knowledge base, providing our agent with the information it needs to identify films.


In [ ]:
films_folder = "films"

!aws s3 sync {films_folder} s3://{bucket_name}/

Now we'll synchronize our uploaded film data with the knowledge base. Amazon Bedrock will:
- Generate vector embeddings from our film documents
- Create semantic indexes for efficient retrieval
- Prepare the data for RAG (Retrieval Augmented Generation)

This synchronization transforms our text-based film information into a format optimized for AI-powered searching. Once complete, our agent will be able to quickly find and retrieve relevant film information based on video content analysis.

Let's initiate the synchronization:


In [ ]:
# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

Let's get the knowledge base ARN - we'll need this unique identifier later when connecting our agent to our film information:


In [ ]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

Let's configure how our agent should use the knowledge base. These instructions will tell the agent to reference this database when it needs to verify film information against its video analysis:


In [ ]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': """Use this knowledge base when you need to look up title, director, and plot of a film"""
}

## Creating Agent

And now let's create and configure the film agent in Amazon Bedrock. 

Amazon Bedrock Agents offers you the ability to build and configure autonomous agents in your application.  With agents, you can automate tasks for your customers and answer questions for them. 

In this particular task we will create an instruction to analyze film videos or deriative of film videos and match it to the correct film title and identify key cast memebers.

In [ ]:
agent_description = """You are a film analyst. You job is analyzing shorts or derivatives of 
films, like promo or trailer. Finding the matching film and identify key cast memebers."""

agent_instruction = """
Your task is to analyze film videos or deriative of film videos and match it to the correct
film title and identify key cast memebers.

Your capabilities include:
<capabilities>
- Matching video to correct film title
- Identify key cast members
</capabilities>

Response style:
<Response style>
- Be helpful and solution-oriented
- Use clear, non-technical language
- Maintain natural conversation flow
- Be concise yet informative
- do not add extra information not required by the user
</Response style>
"""

film_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn],
    code_interpretation=False
)
time.sleep(20)
film_agent

### Associating knowledge base
Now that we've created the agent, let's associate the previously created knowledge base to it.

In [ ]:
agents.associate_kb_with_agent(
    film_agent[0],
    kb_config['kb_instruction'],
    kb_config['kb_id']
)
time.sleep(20)

## Test Knowledge Base W/ Agent

Let's test our Knowledge Base with Agent we have just created.

To do this, we will need to configure knowledge base search and establish detailed search parameters, including vector search configuration, implicit filtering, model selection and others. 

This configuration optimizes the knowledge base search for film identification by enabling filtering by director and using hybrid search to improve accuracy. The session state will be passed to the agent during invocation to control its behavior and search capabilities. 

In [ ]:
session_state = {
    'knowledgeBaseConfigurations': [
            {
                'knowledgeBaseId': kb_config['kb_id'],
                'retrievalConfiguration': {
                    'vectorSearchConfiguration': {
                        'implicitFilterConfiguration': {
                            'metadataAttributes': [
                                {
                                    'description': 'this is the name of the director',
                                    'key': 'Director',
                                    'type': 'STRING'
                                },
                            ],
                            'modelArn': "anthropic.claude-3-5-sonnet-20241022-v2:0"
                        },
                        'numberOfResults': 1,
                        'overrideSearchType': 'HYBRID'
                    }
                }
            },
        ]
}

In the following step we will test the agent's ability to search the knowledge base for film information. 

Let's invoke the agent with a specific query about films directed bu Curtis Clark.

In [ ]:
%%time
import uuid

response = agents.invoke(
    input_text=f"which film is directored by Curtis Clark", 
    agent_id=film_agent[0], 
    enable_trace=True,
    session_id=str(uuid.uuid4()),
    session_state=session_state
)
print("====================")
print(response)

In [ ]:
%store knowledge_base_name
%store kb_config
%store agent_instruction
%store agent_name
%store video_analysis

### Creating Lambda

Now let's create a Lambda function that will be used by the agent to detect celebrities (actors) in video clips, which is essential for identifying films based on the appearance of known cast members.

We will use Amazon Rekognition that makes it easy to automatically recognize tens of thousands of well-known personalities in images and videos using machine learning, as well as Amazon DynamoDB for retrieving additional information about detected celebrities. 

In [ ]:
%%writefile detection.py
import boto3
import json
import os
import time
from boto3.dynamodb.conditions import Key, Attr

## DynamoDB parameters
dynamodb_resource = boto3.resource('dynamodb')
cast_table = os.getenv('cast_table')
cast_pk = os.getenv('cast_pk')

## Rekognition parameters
rek_client = boto3.client('rekognition')

def get_named_parameter(event, name):
    try:
        return next(item for item in event['parameters'] if item['name'] == name)['value']
    except StopIteration:
        raise ValueError(f"Required parameter '{name}' not found in event")
        
def get_cast_member(cast_id):
    try:
        table = dynamodb_resource.Table(cast_table)
        key_expression = Key(cast_pk).eq(cast_id)
        query_data = table.query(
                KeyConditionExpression=key_expression
            )
        return query_data['Items']
    except Exception:
        print(f'Error querying table: {cast_table}.')

def start_celebrity_detection(bucket, video_key):
    response = rek_client.start_celebrity_recognition(
        Video={
            'S3Object': {
                'Bucket': bucket,
                'Name': video_key
            }
        }
    )
    return response['JobId']

def get_celebrity_detection_results(job_id):
    response = rek_client.get_celebrity_recognition(JobId=job_id)
    return response

def extract_bucket_key(video_s3_path):
    path = video_s3_path[5:]  # Remove 's3://'
    bucket, key = path.split('/', 1)  # Split into bucket and key   
    return bucket, key

def detect_key_figures(video_s3_path):
    bucket, key = extract_bucket_key(video_s3_path)
    
    job_id = start_celebrity_detection(bucket, key)
    print(f"Started celebrity detection job: {job_id}")

    while True:
        response = get_celebrity_detection_results(job_id)
        status = response['JobStatus']
        
        if status in ['SUCCEEDED', 'FAILED']:
            print("JOB COMPLETE....")
            break
        
        print("Job in progress...")
        time.sleep(10)

    unique_celebrities = {}  # Dictionary to store unique celebrities

    if status == 'SUCCEEDED':
        for celebrity in response['Celebrities']:
            celeb = celebrity['Celebrity']
            
            # Only process celebrities with 95%+ confidence
            if celeb['Confidence'] >= 95.0:
                celeb_id = celeb['Id']
                
                # Store or update celebrity info only if not already stored
                if celeb_id not in unique_celebrities:
                    celebrity_info = {
                        'name': celeb['Name'],
                        'confidence': celeb['Confidence'],
                        'id': celeb_id,
                        'first_appearance': celebrity['Timestamp']
                    }
                    
                    # If you have additional celebrity info in DynamoDB
                    try:
                        query_items = get_cast_member(celeb_id)
                        if query_items:
                            celebrity_info.update(query_items[0])
                    except Exception as e:
                        print(f"Error fetching additional celebrity info: {str(e)}")
                    
                    unique_celebrities[celeb_id] = celebrity_info
    else:
        print("Detection failed....")

    # Convert the dictionary values to a list for the final output
    final_output = list(unique_celebrities.values())
    return final_output

def populate_function_response(event, response_body):
    return {
        'response': {
            'actionGroup': event['actionGroup'],
            'function': event['function'],
            'functionResponse': {
                'responseBody': {
                    'TEXT': {
                        'body': str(response_body)
                    }
                }
            }
        }
    }
    
def lambda_handler(event, context):
    print(event)
    
    function = event.get('function', '')
    parameters = event.get('parameters', [])
    video_s3_path = get_named_parameter(event, "video_s3_path")

    if function == 'detect_key_figures':
        result = detect_key_figures(video_s3_path)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

We will also need to define a function schema for the celebrity detection action. 

This schema serves as a contract between the agent and the Lambda function, defining what the function does and what inputs it requires. The agent will use this schema to understand when and how to call the function, and to validate that it has all the necessary information before making the call. This structured approach ensures reliable communication between the agent and the Lambda function.

In [ ]:
functions_def = [
    {
        "name": "detect_key_figures",
        "description": """Detect key figures (celebrity, cast member) from a video
        and retrieve information about their position and team""",
        "parameters": {
            "video_s3_path": {
                "description": "S3 location of the video (e.g: s3://......)",
                "required": True,
                "type": "string"
            }
        }
    }
]

### Creating action group and attaching to the agent
Now it's time to add this Lambda function and the function details as an action group for this agent and prepare it.

In [ ]:
agents.add_lambda_action_group_with_rek(
    agent_name=agent_name,
    lambda_function_name=lambda_name,
    source_code_file="detection.py",
    agent_functions=functions_def,
    agent_action_group_name="key_figure_detection_actions",
    agent_action_group_description="Functions to identify key figures and look up cast members and their role from the video",
    env_args=env_args
)
time.sleep(30)

### Loading DynamoDB

The following step populates the DynamoDB table with information about cast members that can be used to enrich the celebrity detection results. When the Lambda function detects a celebrity in a video, it can look up additional information about that person, such as what film they appeared in and what role they played, which is crucial for accurate film identification.

In [ ]:
# Open and read the JSONL file
with open('cast_members.jsonl', 'r') as file:
    table_items = [json.loads(line.strip()) for line in file]

agents.load_dynamodb(cast_table, table_items)

Now let's test the DynamoDB query functionality for cast member lookup !

We will use the specific celebrity ID ('4kn3Xu8r') to look up:

In [ ]:
resp = agents.query_dynamodb(
    cast_table, cast_pk, '4kn3Xu8r'
)
resp

This test confirms that the DynamoDB table is properly populated and can be queried to retrieve cast member information. The Lambda function will use this same query mechanism to enrich celebrity detection results with film and role information, which is essential for the agent to accurately identify films based on the actors that appear in them.

In [ ]:
%store cast_table
%store cast_pk
%store film_video_s3_path

### Test Agent

Let's configure the session state for the agent with video analysis data. 

This configuration provides the agent with all the information it needs to identify a film:
1. The video analysis data for understanding the content
2. The video S3 path for celebrity detection
3. Access to the knowledge base for film information lookup

By including both the analysis results and the S3 path, the agent can use multiple approaches to identify the film - matching the plot details against the knowledge base and detecting celebrities in the video itself.

In [ ]:
session_state = {
    'promptSessionAttributes': {
        "<video_analysis>": json.dumps(video_analysis),
        "<video_s3_path>": film_video_s3_path,
    },
    'knowledgeBaseConfigurations': [
            {
                'knowledgeBaseId': kb_config['kb_id'],
                'retrievalConfiguration': {
                    'vectorSearchConfiguration': {
                        'implicitFilterConfiguration': {
                            'metadataAttributes': [
                                {
                                    'description': 'this is the name of the director',
                                    'key': 'Director',
                                    'type': 'STRING'
                                },
                            ],
                            'modelArn': "anthropic.claude-3-5-sonnet-20241022-v2:0"
                        },
                        'numberOfResults': 1,
                        'overrideSearchType': 'HYBRID'
                    }
                }
            },
        ]
}

And finally, let's test the complete film identification workflow with the agent !

The agent will perform several actions:
   - Step 1: Analyzing the request and planning the approach
   - Step 2: Searching the knowledge base for films matching the plot details
   - Step 3: Deciding to verify with celebrity detection
   - Step 4: Calling the celebrity detection function and analyzing results
   - Final response: Shows the agent's conclusion that the clip is from "Meridian" (2016), confirmed by both plot details and the appearance of actors Kevin Kilner and Reid Scott.

This test demonstrates the agent's ability to combine multiple sources of information - knowledge base search and celebrity detection - to accurately identify a film from a video clip. The execution trace shows the agent's reasoning process, making the identification process transparent and explainable.

In [ ]:
%%time
response = agents.invoke(
    input_text=f"""
    Here is a clip extraction:
    $prompt_session_attributes.video_analysis$

    if needed, here is the s3 location of the video:
    $prompt_session_attributes.video_s3_path$
    
    can you tell me which film is this clip from?
    """, 
    agent_id=film_agent[0],
    enable_trace=True,
    session_id=str(uuid.uuid4()),
    session_state=session_state
)
print("====================")
print(response)

### Create alias

As you can see, you can use your agent with the `TSTALIASID` to complete tasks. 
However, for multi-agents collaboration it is expected that you first test your agent and only use it once it is fully functional. 
Therefore to use an agent as a sub-agent in a multi-agent collaboration you first need to create an agent alias and connect it to a new version. 

Since we've tested and validated our agent, let's now create an alias for it:

In [ ]:
film_agent_alias_id, film_agent_alias_arn = agents.create_agent_alias(
    film_agent[0], 'v2'
)

### Saving Information

Save all the important information about the film agent and its resources for future use in other notebooks or sessions.

In [ ]:
film_agent_arn = agents.get_agent_arn_by_name(agent_name)
film_agent_id = film_agent[0]
film_kb = knowledge_base_name
film_agent_name = agent_name
film_kb_id = kb_config['kb_id']
film_kb_arn = kb_arn
film_kb_description = knowledge_base_description
film_kb_name = knowledge_base_name
film_video_analysis = video_analysis

%store film_agent_alias_id
%store film_agent_alias_arn
%store film_agent_arn
%store film_agent_id
%store film_kb
%store film_agent_name
%store film_kb_id
%store film_kb_arn
%store film_kb_description
%store film_kb_name

%store film_video_analysis
%store film_video_s3_path

### Clean Up

In [ ]:
# agents.delete_agent(agent_name)

In [ ]:
# kb.delete_kb(knowledge_base_name)

In [ ]:
# agents.delete_lambda(lambda_function_name=lambda_name)

In [ ]:
# #delete BDA project
# response = bda_client.delete_data_automation_project(
#     projectArn=video_project_arn
# )